# Entity Matching Songs and Tracks

# Introduction

This IPython notebook explains the workflow of matching two tables using *py_entitymatching*. Our goal is to come up with a workflow to match songs and tracks from Songs.csv and Tracks.csv. Specifically, we want to maximize F1. The datasets contain information about the songs, artists and year they were performed.

First, we need to import *py_entitymatching* package and other libraries as follows:

In [ ]:
import sys

import py_entitymatching as em
import pandas as pd
import os
import re

In [ ]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

# 1. Read the input tables

We begin by loading the input tables A and B from songs.csv and tracks.csv respectively.

In [ ]:
# Set the path of the ipython notebook as the current path
os.getcwd()
orig_dir = os.getcwd()
os.chdir(orig_dir)

# Get the paths
path_A = '../DATA/songs.csv'
path_B = '../DATA/tracks.csv'

Then, we read the input tables.

In [ ]:
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')

In [ ]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

In [ ]:
A.head(2)

In [ ]:
B.head(2)

# 2. Downsample the input tables
As the input tables are large we downsample the input tables to obtain sample_A and sample_B from A and B respectively.

In [ ]:
sample_A, sample_B = em.down_sample(A, B, size=5000, y_param=1, show_progress=True)
print('Number of tuples in sample_A: ' + str(len(sample_A))) 
print('Number of tuples in sample_B: ' + str(len(sample_B)))

# 3. Block tables to get candidate set

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.

For the matching problem at hand, we know that two songs with different titles or different artist names will not match. We identified all (song, track) pairs such that song.title shared at least one word with track.song, and song.artist_name shared at least one word with track.artists. We noticed that Magellan’s overlap blocker only performs blocking on a single column, so for convenience, we initially performed black-box blocking.

In [ ]:
# Define functions for black-box blocking
def match(ltup, rtup):
    '''
    Returns True if (ltup, rtup) should be dropped, or False if (ltup, rtup) is
    a candidate. Called by bb_block().
    '''
    l_song_withstop = str(ltup['title']).lower().split()
    r_song_withstop = str(rtup['song']).lower().split()
    l_artist_withstop = str(ltup['artist_name']).lower().split()
    r_artist_withstop = str(rtup['artists']).lower().replace('+', ' ').split()
    stopwords = ('the', 'a')
    l_song = set(w for w in l_song_withstop if w not in stopwords)
    r_song = set(w for w in r_song_withstop if w not in stopwords)
    l_artist = set(w for w in l_artist_withstop if w not in stopwords)
    r_artist = set(w for w in r_artist_withstop if w not in stopwords)

    # If no overlap among artists or no overlap among songs, then drop
    return l_artist.isdisjoint(r_artist) or l_song.isdisjoint(r_song)

def bb_block(sample_A, sample_B):
    '''
    Returns a DataFrame of candidate pairs by performing black-box blocking.
    '''
    sample_B['artists'] = sample_B['artists'].str.replace('+', ' + ')

    # Create black box blocker
    bb = em.BlackBoxBlocker()
    bb.set_black_box_function(match)
    bbC = bb.block_tables(sample_A, sample_B,
                        l_output_attrs=['id', 'title', 'artist_name', 'year'],
                        r_output_attrs=['id', 'title', 'year', 'episode', 'song', 'artists'],
    )

    return bbC

sample_A = em.read_csv_metadata('../DATA/sample_A.csv', key='id')
sample_B = em.read_csv_metadata('../DATA/sample_B.csv', key='id')

## a. Apply the blocker on the downsampled tables.

The number of tuple pairs considered for matching is reduced to 5223 (from 20900000):

In [ ]:
C = bb_block(sample_A, sample_B)
len(C)

## b. Debugging the blocker output

We would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in *py_entitymatching* as follows:

In [ ]:
# Debug blocker output
dbg = em.debug_blocker(C, sample_A, sample_B, attr_corres=[('title', 'song'),('artist_name', 'artists')], output_size=20)
dbg.head()

## c. Apply the modified blocker on the downsampled tables.

The above blackbox approach was prohibitively slow, taking roughly 20 minutes to run on the downsampled tables. Clearly, black-box blocking would not scale to the entirety of the Song and Track tables during the production stage. Hence we modified our blocking sequence to include the overlap blocker.

In [ ]:
# Display first few tuple pairs from the debug_blocker's output
def overlap_block(sample_A, sample_B):
    '''
    Returns a DataFrame of candidate pairs by performing overlap blocking.
    Performs overlap blocking on title, then overlap blocking on artist, then
    black-box blocking to guarantee the same results as bb_block().
    '''
    # many artist names in sample_B are something like 
    # 'budda+wc+ice cube+mack 10+westside connection'
    # change this to 'budda + wc + ice cube + mack 10 + westside connection'
    sample_B['artists'] = sample_B['artists'].str.replace('+', ' + ')
    
    # Create overlap blocker
    # If no overlap among artists or no overlap among songs, then drop
    ob = em.OverlapBlocker()
    ob.stop_words = ['a', 'the']
    ob.regex_punctuation = re.compile(r'')
    obC1 = ob.block_tables(sample_A, sample_B, 'title', 'song',
                        l_output_attrs=['id', 'title', 'artist_name', 'year'],
                        r_output_attrs=['id', 'title', 'year', 'episode', 'song', 'artists'],
                        rem_stop_words=True,
    )
    ob.regex_punctuation = re.compile(r'\+')
    obC2 = ob.block_candset(obC1, 'artist_name', 'artists', rem_stop_words=True)

    # Create black box blocker
    # Necessary because overlap blocker behaves incorrectly when words contain Unicode characters
    bb = em.BlackBoxBlocker()
    bb.set_black_box_function(match)
    bbC = bb.block_candset(obC2)
    return bbC

C = overlap_block(sample_A, sample_B)
len(C)

We debugged the blocker output again to check if the current blocker sequence is dropping any potential matches.

In [ ]:
# Display first few rows from the debugger output
dbg = em.debug_blocker(C, sample_A, sample_B, attr_corres=[('title', 'song'),('artist_name', 'artists')], output_size=20)
dbg.head(3)

We observe that the current blocker sequence does not drop obvious potential matches, and we can proceed with the matching step now. 

## d. Stop modifying the blocker
We decide to stop blocking when we observed very less potential matches were dropped and we had developed a blocker sequence which was reasonably fast i.e, it took less than 30s to perform blocking on the downsampled data. 

# 4. Sampling and labeling the candidate set

First, we randomly sample 500 tuple pairs for labeling purposes.

In [ ]:
# Sample candidate set
S = em.sample_table(C, 500)
S.head()

Next, we label the sampled candidate set. Specifically we would enter 1 for a match and 0 for a non-match.

In [ ]:
# Label S
# G = em.label_table(S, label_column_name='gold_labels')

For the purposes of this notebook, we will load in a pre-labeled dataset (of 500 tuple pairs).

In [ ]:
G = em.read_csv_metadata('../DATA/G.csv', key='_id')
G.head(5)

# 5. Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher, and the evaluation set is used to evaluate the selected matcher on unseen data.

In [ ]:
# Split S into development set (I) and evaluation set (J)
# G.head(10)
# IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
# I = IJ['train']
# J = IJ['test']

#for the purpose of this notebook read the already saved train and test files 
I = em.read_csv_metadata('../DATA/I.csv', key='_id', ltable=sample_A, rtable=sample_B, fk_ltable='ltable_id', fk_rtable='rtable_id')
J = em.read_csv_metadata('../DATA/J.csv', key='_id', ltable=sample_A, rtable=sample_B, fk_ltable='ltable_id', fk_rtable='rtable_id')

# 6. Creating a set of learning-based matchers

The six learning-based matcher types in Magellan are Decision Tree, Random Forest, SVM, Naive Bayes, Logistic Regression, and Linear Regression.

In [ ]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

# 7. Selecting the best matcher using I. This step include:

### a. Creating features

Next, we need to create a set of features for the development set. *py_entitymatching* provides a way to automatically generate features based on the attributes in the input tables. We automatically generate the features by providing the correspondence between the two tables.

In [ ]:
# Generate features
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()
atypes1 = em.get_attr_types(sample_A)
atypes2 = em.get_attr_types(sample_B)
match_c = em.get_attr_corres(sample_A, sample_B)
match_c['corres'] = [('title', 'song'),('artist_name', 'artists')]
match_f = em.get_features(sample_A, sample_B, atypes1, atypes2, match_c, match_t, match_s)
feature_table = match_f

In [ ]:
# List the names of the features generated
feature_table['feature_name']

### b. Convert the I into a set of feature vectors using F

In [ ]:
H = em.extract_feature_vecs(I, feature_table=feature_table, attrs_before= ['_id', 'ltable_id', 'rtable_id'], attrs_after='gold_labels', show_progress=False)
# Display first few rows
H.head(3)

### c. Filling in the missing values if any
We do not have any missing values as the tuples with missing values are filtered out by the blocker.


### d. Selecting the best matcher using cross-validation
Now, we select the best matcher using k-fold cross-validation. We used ten fold cross validation and use 'F1' metric to select the best matcher.

In [ ]:
# Select the best ML matcher using CV
attrs_to_be_excluded = ['_id', 'ltable_id', 'rtable_id', 'gold_labels']
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=attrs_to_be_excluded,
        k=10,
        target_attr='gold_labels', metric='f1', random_state=0)
result['cv_stats']

### e. Debugging matcher

We observe that the best matcher is Random Forest. However it is still not maximizing F1. We debug the matcher to see what might be wrong.
To do this, first we split the feature vectors into train and test.

In [ ]:
#  Split feature vectors into train and test
UV = em.split_train_test(H, train_proportion=0.5)
U = UV['train']
V = UV['test']

Next, we debug the matcher using GUI. Since the best matcher we found was random forest matcher, we went ahead debugging it.

In [ ]:
# Debug decision tree using GUI
em.vis_debug_rf(rf, U, V, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'],
        target_attr='gold_labels')

After debugging it was found that there were errors in labeling. To rectify this, we looked over the labeling to check for errors and fix them. In the dev set I, there were 5 errors found; all were instances where the tuple pairs were a match (should be 1) but were marked as not a match (mistakenly labeled as 0). After fixing the labeling the matcher was run again.

In [ ]:
# Select the best ML matcher using CV
attrs_to_be_excluded = ['_id', 'ltable_id', 'rtable_id', 'gold_labels']
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=attrs_to_be_excluded,
        k=5,
        target_attr='gold_labels', metric='f1', random_state=0)
result['cv_stats']

Now, observe the best matcher which is achieving a better F1. 

#  8. Evaluating the matching output

Evaluating the matching outputs for the evaluation set typically involves the following four steps:
1. Converting the evaluation set to feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

### a. Converting the evaluation set to  feature vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set).

In [ ]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='gold_labels', show_progress=False)

### b. Training the selected matcher

Now, we train the matcher using all of the feature vectors from the development set. We have found random forest as the selected matcher.

In [ ]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'], 
       target_attr='gold_labels')

### c. Predicting the matches

Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [ ]:
# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold_labels'], 
              append=True, target_attr='predicted', inplace=False)

### d. Computing the accuracy

Finally, we evaluate the accuracy of predicted outputs.

In [ ]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold_labels', 'predicted')
em.print_eval_summary(eval_result)